### Imports:

In [1]:
import numpy as np
import datetime as dt
import time
import re
import requests
import matplotlib.pyplot as plt
from pandas import Series, DataFrame, read_json, to_datetime, to_numeric
#from pandas import Timedelta

### Exemplos "originais" (via Telegram):

In [2]:
dados = read_json('https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=15m&limit=340')
dados2 = read_json('https://api.binance.com/api/v3/klines?symbol=VETUSDT&interval=15m&startTime=1628125200000&endTime=1628989200000&limit=1000')

### Exemplos interativos:

In [ ]:
symbol = 'BTCUSDT'
interval='15m'
limit='340'

dados3 = read_json('https://api.binance.com/api/v3/klines?symbol={}&interval={}&limit={}'.format(
   symbol, interval, limit)
)

dados3.head()

In [ ]:
fig = dados3[4].plot(figsize = (16,8), grid=True, fontsize = 18, linewidth=2.0)
plt.title(f'{symbol} Close values over the time', fontdict = {'fontsize' : 25})


print(dados3[4].min())
print(dados3[4].max())

print((dados3[4].max() - dados3[4].min())/(dados3[4].max()))


In [ ]:
symbol = 'VETUSDT'
interval='15m'
limit='1000'

data_inicio = '2021-08-04'
horario_inicio = '22:00'

data_fim = '2021-08-14'
horario_fim = '22:00'

startTime = int(dt.datetime.fromisoformat(data_inicio+'T'+horario_inicio).timestamp()) * 1000
endTime = int(dt.datetime.fromisoformat(data_fim+'T'+horario_fim).timestamp()) * 1000

dados4 = read_json('https://api.binance.com/api/v3/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}'.format(
   symbol, interval, startTime, endTime, limit)
)

dados4.head()

### Coletando dados de um range largo de datas e exportando resultados:

In [5]:
symbols = ['BTC','ETH','BNB']
#symbols = ['DOGE','CHZ','MATIC','XRP','BNB','XLM','THETA','VET','ETC','BTT','FIL','ADA','LTC','UNI','TRX','ENJ','DOT','EOS','ETH','BCH','ATOM','LINK','AXS','NEO','BTC','XTZ','MKR','AAVE','XMR','FTT','ALGO','SOL']


In [6]:
def gerar_csv_dados_cripto(symbol, interval, limit, data_inicio=None, horario_inicio=None, data_fim=None, horario_fim=None, log=True):
    if(interval == '1m'):
        delta = dt.timedelta(hours=16)
    elif(interval == '3m'):
        delta = dt.timedelta(days=2)
    elif(interval == '5m'):
        delta = dt.timedelta(days=3)
    elif(interval == '15m'):
        delta = dt.timedelta(days=10)
    elif(interval == '30m'):
        delta = dt.timedelta(days=20, hours=20)
    elif(interval == '1h'):
        delta = dt.timedelta(days=40)
    elif(interval == '2h'):
        delta = dt.timedelta(days=80)
    elif(interval == '4h'):
        delta = dt.timedelta(days=160)
    elif(interval == '6h'):
        delta = dt.timedelta(days=240)
    elif(interval == '8h'):
        delta = dt.timedelta(days=320)
    elif(interval == '12h'):
        delta = dt.timedelta(days=480)
    elif(interval == '1d'):
        delta = dt.timedelta(days=1000)
    elif(interval == '3d'):
        delta = dt.timedelta(days=3000)
    elif(interval == '1w'):
        delta = dt.timedelta(weeks=1000)
    elif(interval == '1M'):
        delta = dt.timedelta(weeks=4000)
    else:
        raise ValueError('intervalError')
        
    if(horario_inicio == None):
        horario_inicio = '00:00:00'
    if(horario_fim == None):
        #horario_fim = dt.datetime.now()
        #horario_fim = str(horario_fim.hour)+':'+str(horario_fim.minute)+':'+str(horario_fim.second)
        horario_fim = '23:59:59'
        
    if(limit > 1000):
        limit = 1000
    if(limit <= 0):
        raise ValueError('limitError')
        
    if(data_inicio == None):
        path_dados = 'https://api.binance.com/api/v3/klines?symbol={}&interval={}&limit={}'.format(
            symbol, interval, limit
        )
        dados_atual = read_json(path_dados).iloc[:, :5]
        dados_atual.insert(loc=0, column='Date-Time', value = dados_atual[0].apply(lambda x : dt.datetime.fromtimestamp(x/1000)))
        
        print('Gerando arquivo .csv...')

        dados_atual.columns = ['Date-Time', 'Timestamp', 'Open', 'Max', 'Min', 'Close']
        dados_atual.set_index('Date-Time', inplace=True)

        dados_atual.to_csv(f'dados_{symbol}_{interval}.csv')

        print('Procedimento realizado com sucesso!')
    
        return
    elif((data_inicio != None) and (data_fim != None)):
        path_dados_base = 'https://api.binance.com/api/v3/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}'
        limit = 1000
    else:
        data_fim = dt.datetime.now()
        data_fim = str(data_fim.year)+'-'+str(data_fim.month)+'-'+str(data_fim.day)
        
        path_dados_base = 'https://api.binance.com/api/v3/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}'
        
        limit = 1000

    data_atual = dt.datetime.fromisoformat(data_inicio+'T'+horario_inicio)

    dados_list = list()
    
    primeiro_laco = True

    momento_final = dt.datetime.fromisoformat(data_fim+'T'+horario_fim)
    momento_final_timestamp = int(dt.datetime.timestamp(momento_final)) * 1000
    
    it_count = 1
    
    while(data_atual < momento_final):
        if(log):
            print('Iteracao Atual:', it_count)
            print('Data Atual:', data_atual)

        startTime = int(data_atual.timestamp()) * 1000
        aux_endTime = int( (data_atual + delta).timestamp()) * 1000
        if(aux_endTime < momento_final_timestamp):
            endTime = aux_endTime
        else:
            endTime = momento_final_timestamp

        
        dados_iter = re.findall(r'\[[\d]+,"[\d]+.[\d]+","[\d]+.[\d]+","[\d]+.[\d]+","[\d]+.[\d]+"', requests.get(
            path_dados_base.format(symbol, interval, startTime, endTime, limit)).text
        )
        
        if(primeiro_laco):
            dados_list.extend(dados_iter)
        else:
            dados_list.extend(dados_iter[1:])

        data_atual += delta
        it_count += 1
        
        primeiro_laco = False

    print('Gerando arquivo .csv...')        

    dados_completo = Series(dados_list).str.replace('[\["]', '', regex=True)
    dados_completo = DataFrame([dados_completo[i].split(',') for i in range(len(dados_completo))])
    dados_completo = dados_completo.apply(to_numeric)
    
    dados_completo.insert(0, 'Date-Time', (dados_completo[0]/1000).apply(lambda x : dt.datetime.fromtimestamp(x)))

    dados_completo.columns = ['Date-Time', 'Timestamp', 'Open', 'Max', 'Min', 'Close']
    dados_completo.set_index('Date-Time', inplace=True)

    dados_completo.to_csv(f'dados_{symbol}_{interval}.csv')

    print('Procedimento realizado com sucesso!')
    
    return

In [8]:
time_init = time.time()
gerar_csv_dados_cripto(symbol='BTCUSDT', interval='30m', limit=1000, data_inicio='2021-06-01', log=False)
print('Tempo do procedimento:', time.time() - time_init, 's')

Gerando arquivo .csv...
Procedimento realizado com sucesso!
Tempo do procedimento: 5.530316114425659 s
